### Seq-to-seq coding

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F 
import random 
import numpy as np 
import spacy 
import datasets 
import torchtext
import tqdm 
import evaluate 


In [47]:
seed = 1234 
random.seed(seed) 
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) 
torch.backends.cudnn.deterministic = True

### Dataset

In [48]:
dataset = datasets.load_dataset("bentrevett/multi30k")

In [49]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [50]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"]
)

In [51]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

### Tokenizers

In [52]:
# # Download the model if not already installed
# try:
#     spacy.load("en_core_web_sm")
# except OSError:
#     from spacy.cli import download
#     download("en_core_web_sm")

# # Load the model
# nlp = spacy.load("en_core_web_sm")

In [53]:
# # Download the model if not already installed
# try:
#     spacy.load("de_core_news_sm")
# except OSError:
#     from spacy.cli import download
#     download("de_core_news_sm")

# # Load the model
# nlp = spacy.load("de_core_news_sm")

In [54]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [55]:
string = "What a lovely day it is today"
[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today']

In [56]:
# แปลงให้เป็น Token แล้วทำให้เป็นตัวเล็ก พร้อม ใส่ <sos> กับ <eos>
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens" : en_tokens, "de_tokens" : de_tokens} # เพิ่ม list เข้าไป ซึ่งก็คือ en_tokens และ de_tokens

In [57]:
max_length = 1000
lower = True 
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp" : en_nlp,
    "de_nlp" : de_nlp, 
    "max_length" : max_length,
    "lower" : lower,
    "sos_token" : sos_token,
    "eos_token" : eos_token
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

In [58]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

### Vocabularies

In [59]:
min_freq = 2 
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

In [60]:
en_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 'a',
 '.',
 'in',
 'the',
 'on',
 'man',
 'is',
 'and',
 'of',
 'with',
 'woman',
 ',',
 'two',
 'are',
 'to',
 'people']

In [61]:
en_vocab.get_itos()[9]

'man'

In [62]:
de_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '.',
 'ein',
 'einem',
 'in',
 'eine',
 ',',
 'und',
 'mit',
 'auf',
 'mann',
 'einer',
 'der',
 'frau',
 'die',
 'zwei',
 'einen']

In [63]:
en_vocab.get_stoi()["the"]

7

In [64]:
en_vocab["the"]

7

In [65]:
len(en_vocab), len(de_vocab)

(5893, 7853)

In [66]:
"the" in en_vocab

True

In [67]:
"The" in en_vocab

False

In [68]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]



In [69]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [70]:
en_vocab["The"]

0

In [71]:
en_vocab.get_itos()[0]

'<unk>'

In [72]:
tokens = ["i", "love", "watching", "crime", "show"]

In [73]:
en_vocab.lookup_indices(tokens)

[956, 2169, 173, 0, 509]

In [74]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', '<unk>', 'show']

In [75]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [76]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [77]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [78]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

['<sos>',
 'two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.',
 '<eos>']

In [79]:
# สรุป คือ เราได้สร้าง dictionary แปลงเป็นตัวเลข(index) พร้อม mapไปกับdatasetแต่ละตัว

### เปลี่ยน index ให้เป็น Tensor

เราจะเปลี่ยน object ของ Train_data (ในที่นี้คือ en_ids และ de_ids) ไปเป็น Torch Tensor

In [80]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(type=data_type, columns=format_columns, output_all_columns=True)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

In [81]:
train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [82]:
type(train_data[0]["en_ids"])

torch.Tensor

In [86]:
def get_collect_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids" : batch_en_ids,
            "de_ids" : batch_de_ids
        }
        return batch 
    
    return collate_fn

In [88]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collect_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle
    )
    return data_loader

In [89]:
batch_size = 128 
train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data = get_data_loader(valid_data, batch_size, pad_index)
test_data = get_data_loader(test_data, batch_size, pad_index)

### Build the Model